In [3]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy as sp

from mindquantum.core.gates import X, Y, Z, S, H, RX, RY, RZ, Measure, UnivMathGate, DepolarizingChannel
from mindquantum.core.circuit import Circuit
from mindquantum.simulator import Simulator

In [57]:
def SpinChain(n,J,h):
    X = np.array([[0, 1], [1, 0]],dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]],dtype=complex)
    Z = np.array([[1, 0], [0, -1]],dtype=complex)
    def hopping(P,i):
        assert i < n, "i should be less than n"
        if i == 0 or i == n-1:
            matrix = P
        else:
            matrix = np.eye(2,dtype=complex)
        for j in range(1,n):
            if j == i or j == i+1:
                matrix = np.kron(P,matrix)
            else:
                matrix = np.kron(np.eye(2,dtype=complex),matrix)
        return matrix
    def potential(P,i):
        assert i < n, "i should be less than n"
        if i == 0:
            matrix = P
        else:
            matrix = np.eye(2,dtype=complex)
        for j in range(1,n):
            if j == i:
                matrix = np.kron(P,matrix)
            else:
                matrix = np.kron(np.eye(2,dtype=complex),matrix)
        return matrix
    
    # hopping term
    HoppingX = np.zeros((2**n,2**n),dtype=complex)
    HoppingY = np.zeros((2**n,2**n),dtype=complex)
    HoppingZ = np.zeros((2**n,2**n),dtype=complex)
    if n == 2:
        for i in range(n-1):
            HoppingX += hopping(X,i)*J[0]
            HoppingY += hopping(Y,i)*J[1]
            HoppingZ += hopping(Z,i)*J[2]
    else:
        for i in range(n):
            HoppingX += hopping(X,i)*J[0]
            HoppingY += hopping(Y,i)*J[1]
            HoppingZ += hopping(Z,i)*J[2]
    # potential term
    PotentialX = np.zeros((2**n,2**n),dtype=complex)
    PotentialY = np.zeros((2**n,2**n),dtype=complex)
    PotentialZ = np.zeros((2**n,2**n),dtype=complex)
    for i in range(n):
        PotentialX += potential(X,i)*h[0]
        PotentialY += potential(Y,i)*h[1]
        PotentialZ += potential(Z,i)*h[2]
    return HoppingX+HoppingY+HoppingZ+PotentialX+PotentialY+PotentialZ

def TimeEvolution(H,t):
    eigenv,U = np.linalg.eig(H)
    diag = np.diag(np.exp(-1.j*t*eigenv))
    return U@diag@np.linalg.inv(U)

def IPEA(m,hamil,ground_state,size = 1000, error = 0.0, Evolvef = None):
    n = int(np.log2(hamil.shape[0]))
    sim = Simulator('mqvector',n+1)
    target_bit = [i+1 for i in range(n)]

    phi = 0
    for i in tqdm(range(m-1,-1,-1)):
        if Evolvef is None:
            Ut = TimeEvolution(hamil,2**i)
        else:
            Ut = Evolvef(2**i)
        randmat = np.random.randn(2**n,2**n)+1.j*np.random.randn(2**n,2**n)
        randmat = randmat*error*(2**i)
        Ut = Ut + randmat

        circ = Circuit()
        circ += H.on(0)
        evol_gate = UnivMathGate('Ut',Ut)
        circ += evol_gate.on(target_bit,0)
        Rz = np.array([[1.0, 0.0], [0.0, np.exp(1.j*phi)]],dtype=complex)
        circ += UnivMathGate('Rz',Rz).on(0)
        circ += H.on(0)
        circ += Measure('q0').on(0)
        
        sim.reset()
        sim.set_qs(np.kron(ground_state,[1.0,0.0]))
        result = sim.sampling(circuit=circ,shots=size)
        samples = result.data
        # print(samples)
        try:
            zero = samples['0']
        except:
            zero = 0
        try:
            one = samples['1']
        except:
            one = 0
        if i == 0:
            if zero > one:
                phi = -phi
            else:
                phi = -(phi - np.pi)
        else:
            if zero > one:
                phi = phi/2
            else:
                phi = phi/2 + np.pi/2
    return -phi

In [58]:
n = 8
J = [-1.0,-1.0,-1.5]
h = [1.5,0.0,0.5]
hamil = SpinChain(n,J,h)
ground_energy = np.linalg.eigvalsh(hamil)[0]

hamil = hamil*0.4 + np.identity(hamil.shape[0])*11
ground_state = 0.9*np.linalg.eig(hamil)[1][:,0]+np.sqrt(1-0.9**2)*np.linalg.eig(hamil)[1][:,1]

In [63]:
dU_error = [1e-2,1e-3,1e-4,1e-5,0]
samples = []
for i in tqdm(range(10),desc='samples',colour='blue'):
    energy_error = []
    for e in dU_error:
        estimate_phi = IPEA(m = 11, hamil = hamil, ground_state = ground_state, error = e)
        estimate_energy = (estimate_phi-11)/0.4
        energy_error.append(estimate_energy-ground_energy)
    samples.append(energy_error)

samples: 100%|██████████| 10/10 [03:24<00:00, 20.42s/it]


In [62]:
post_samples = np.array(np.abs(samples)).T
benchmark = []
for i in range(len(dU_error)):
    benchmark.append((np.mean(post_samples[i]),np.std(post_samples[i])))
benchmark = np.array(benchmark)
print('mean:',benchmark[:,0])
print('std:',benchmark[:,1])

mean: [0.55925382 0.0696783  0.00064917 0.00064917 0.00064917]
std: [0. 0. 0. 0. 0.]


# 2. Non-Hermitian

In [87]:
def GainLoss_Hamiltonian(g,delta=1.0,kappa=0.5):
    return np.array([[delta-1.j*g, kappa], [kappa, delta+1.j*g]])

def EP_time_evolution(t):
    t = t*2.0
    U = np.array([[-1.j,2.0],[1.0,0.0]])
    Jordan_t = np.array([[np.exp(-1.j*t), -1.j*np.exp(-1.j*t)*t],[0.0, np.exp(-1.j*t)]])
    return U @ Jordan_t @ np.linalg.inv(U)

delta_t = 2.0

In [71]:
# before EP
gg = [0.2,0.3,0.4]
error = []
for g in gg:
    NHH = GainLoss_Hamiltonian(g)
    gs = np.sqrt(1-0.9**2)*np.linalg.eig(NHH)[1][:,0] + 0.9*np.linalg.eig(NHH)[1][:,1]
    gs = gs/np.linalg.norm(gs)
    gs_e = np.linalg.eig(NHH)[0][1]
    print(gs_e)
    estimate_e = IPEA(m = 10, hamil = NHH*delta_t, ground_state = gs, error = 0)
    print("estimate e:",estimate_e/2.0)
    error.append(np.abs(estimate_e/2-gs_e))
error

(0.541742430504416-5.4905503128247676e-17j)


100%|██████████| 10/10 [00:00<00:00, 750.91it/s]


estimate e: 0.543029198911517
(0.5999999999999999-4.85722573273506e-17j)


100%|██████████| 10/10 [00:00<00:00, 802.83it/s]


estimate e: 0.5982525072754
(0.7+9.71445146547012e-17j)


100%|██████████| 10/10 [00:00<00:00, 666.70it/s]

estimate e: 0.6994952392758524


[0.0012867684071010688, 0.0017474927245998417, 0.0005047607241475127]

In [74]:
# after EP
gg = [0.6,0.7,0.8]
error = []
for g in gg:
    NHH = GainLoss_Hamiltonian(g)
    gs = np.sqrt(1-0.9**2)*np.linalg.eig(NHH)[1][:,0] + 0.9*np.linalg.eig(NHH)[1][:,1]
    gs = gs/np.linalg.norm(gs)
    gs_e = np.linalg.eig(NHH)[0][1]
    print(gs_e)
    estimate_e = IPEA(m = 10, hamil = NHH*delta_t, ground_state = gs, error = 0)
    print("estimate e:",estimate_e/2.0)
    error.append(np.real(estimate_e/2+gs_e))
error

(1+0.3316624790355398j)


100%|██████████| 10/10 [00:00<00:00, 722.16it/s]


estimate e: -1.5707963267948966
(1+0.48989794855663543j)


100%|██████████| 10/10 [00:00<00:00, 768.84it/s]


estimate e: -1.5707963267948966
(1+0.6244997998398398j)


100%|██████████| 10/10 [00:00<00:00, 833.16it/s]

estimate e: -1.5707963267948966


[-0.5707963267948966, -0.5707963267948966, -0.5707963267948966]

In [89]:
# EP
gs1 = np.array([-1.j,1.0])/np.sqrt(2)
gs2 = np.array([1.0,0.0])
gs = 0.9*gs1 + np.sqrt(1-0.9**2)*gs2
gs = gs/np.linalg.norm(gs)
estimate_e = IPEA(m = 10, hamil = NHH*delta_t, ground_state = gs, error = 0, Evolvef=EP_time_evolution)
print("estimate e:",estimate_e/2.0)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 729.41it/s]

estimate e: 1.1780972450961724


# 3. Floquet

In [50]:
from functools import partial
def EvolvedU(t,dU):
    return np.linalg.matrix_power(dU,t)

In [ ]:
# non-T
dU = np.array([[-0.599986 - 0.7382 *1.j, 0, -0.0527668 + 0.303797 *1.j, 0], 
               [0, -0.814197 - 0.338483 *1.j, 0, -0.0807238 + 0.464755 *1.j], 
               [-0.215087 + 0.22094 *1.j, 0, -0.949803 - 0.0528952 *1.j, 0], 
               [0, -0.329044 + 0.337998 *1.j, 0, -0.751733 - 0.460852 *1.j]])

gs = np.array([-0.612372, -0.353553, 0.353553, 0.612372])

estimate_e = IPEA(m = 10, hamil = dU, ground_state = gs, error = 0, Evolvef=partial(EvolvedU, dU=dU))
print("estimate e:",estimate_e/2.0)

100%|██████████| 10/10 [00:00<00:00, 903.05it/s]

estimate e: 1.0768545130957201


In [55]:
# T
dU = np.array([[-0.462832 - 0.518753 *1.j, 0, 0. + 0.718806 *1.j, 0], 
               [0, -0.439416 + 0.356942 *1.j, 0, 0. + 0.824322 *1.j], 
               [0. + 0.718806 *1.j, 0, -0.462832 + 0.518753 *1.j, 0], 
               [0, 0. + 0.824322 *1.j, 0, -0.439416 - 0.356942 *1.j]])

gs = np.array([-0.612372, -0.353553, 0.353553, 0.612372])

estimate_e = IPEA(m = 10, hamil = dU, ground_state = gs, error = 0, Evolvef=partial(EvolvedU, dU=dU))
print("estimate e:",estimate_e/9.0+1)

100%|██████████| 10/10 [00:00<00:00, 769.40it/s]

estimate e: 1.2277109258461352
